In [2]:
import chromadb
#https://github.com/chroma-core/chroma/blob/main/chromadb/experimental/density_relevance.ipynb

client =  chromadb.PersistentClient('./chroma_docs')
col_raw = client.get_collection('documents_raw')
#col_raw = client.get_collection('documents_summary')

data = col_raw.get()

print(data.keys())
len(data['documents'])

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data', 'included'])


3180

In [35]:
results = col_raw.query(query_texts="what is graph rag?",n_results=5)
print(results.keys())
results['documents']


dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data', 'included'])


[['9.4 GraphRAG in Industry\n\nIn this section, we mainly focus on industrial GraphRAG systems. These systems are characterized by their reliance on industrial graph database systems or their focus on large-scale graph data, details of which are as follows.\n\n• GraphRAG (by Microsoft)10: The system uses LLMs to construct entity-based knowledge graphs and pre-generate community summaries of related entity groups, which enables the capture of both local and global relationships within a document collection, thereby enhancing Query-Focused Summarization (QFS) task [32]. The project can also utilize open-source RAG toolkits for rapid implementation, such as LlamaIndex11, LangChain12, etc.\n\n• GraphRAG (by NebulaGraph)13: The project is the first industrial GraphRAG system, which is developed by NebulaGraph Corporation. The project integrates LLMs into the NebulaGraph database, which aims to deliver more intelligent and precise search results.\n\n• GraphRAG (by Antgroup)14: The framework 

In [84]:
import litellm
import os
from json_repair import loads


os.environ["OPENAI_API_KEY"] = "any"
BASE_URL = "http://localhost:8080/"
MODEL_NAME = "gpt-4o"
litellm.set_verbose = False

sys_msg = "You are an expert question designer."

def completion(msg,sys_msg=sys_msg):
    messages = [{"role":"system","content":sys_msg},
                     {"role": "user", "content":msg }]
    r= litellm.completion(
        model=MODEL_NAME,
        messages=messages,
        base_url=BASE_URL,
        safety_settings=[
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ], 
        response_format={"type": "json_object"}
    
    )
     
    j= r.choices[0].message.content
    return loads(j)


create_base_question_prompt ="""
From the following piece of TEXT create questions with answers. questions has to be from various parts of the text.
'''TEXT
{doc}
'''
Your response is in JSON format with these fileds.
{{
Question: a question from the TEXT 
Answer: an answer to the question
Number: the question number start from 0
}}

You have to create 10 questions.
"""

evolve_question_prompt= """
You are given a BASE_Question and its answer. You have to create short different questions from the BASE_Question by changing it.
BASE_Question: {question}
Answer: {answer}
Your response is in JSON array with these fileds.
'''json
Reasoning: an array of questions focuing on improving of the reasoning of the topic.
Deepening: an array of questions each goes deeper into the topic.
Constraint: an array of questions that has some constraints.
Complicate:  an array of questions that complicate the input.
Evolving:  an array of questions that each is completly different from the BASE_Question but it must be in the same topic.
Abstract: an array of questions that focus on abstract concepts.
'''

each array has 5 element and questions are not even remotely similar to each other.
"""

answer_prompt="""Answer to this question based on the following text in a JSON format like:
{{"Answer": the answer to the question based on the text or inferred from the text
"Refrences": from which parts of the text Answer is extracted
"Answerable": true if you can extract the answer from the text or false if you can not
}}

##################
Question:
{question}

##################
Text:
{text}

##################
Output:
"""

#Difference: A number between 0-10 indicating diffence of this set of questions to the previous sets.
#

In [85]:
import pandas as pd
communities_path  = '/home/cip/ce/ix05ogym/Majid/LLM/GraphRag/elec_graph/output/create_final_community_reports.parquet'
communities_reports = pd.read_parquet(communities_path)
communities_reports_content = communities_reports['full_content']
communities_reports_content

0     # Soldering and Electronics Assembly\n\nThis c...
1     # Flux and Solderability\n\nThis community rev...
2     # Assembly and its Associated Faults\n\nThis c...
3     # Component Assembly and Manufacturing\n\nThis...
4     # Electrical Connections and Circuits\n\nThis ...
                            ...                        
68    # BS 4584 and Printed Circuit Board Laminates\...
69    # Solderability and Electronics Assembly\n\nTh...
70    # Printed Wiring Boards and Related Processes\...
71    # Metal and Solder Joints in Electronics\n\nTh...
72    # Soldering and Electronics Assembly\n\nThis c...
Name: full_content, Length: 73, dtype: object

In [86]:
import chromadb

client = chromadb.PersistentClient('chroma_docs')
col = client.get_or_create_collection('electronic')
data = col.get(limit=1)
#doc = data['documents'][1]

base_questions = completion(create_base_question_prompt.format(doc=communities_reports_content[0]))
display(base_questions)


[{'Question': 'What is the primary function of soldering in the context of electronics assembly?',
  'Answer': 'Soldering is the process of joining metal parts together, specifically in the context of electronics assembly, by melting and flowing solder into the joint to create a strong and reliable bond.',
  'Number': 0},
 {'Question': 'What are the two main materials used in the soldering process?',
  'Answer': 'The two main materials used in soldering are solder and flux.',
  'Number': 1},
 {'Question': 'What is the purpose of flux in soldering?',
  'Answer': 'Flux removes oxides and contaminants from the surfaces of metal parts, which improves the wetting and flow of the solder, leading to a better bond.',
  'Number': 2},
 {'Question': 'Why is safety a critical aspect of soldering?',
  'Answer': 'Soldering involves working with hot materials and potentially hazardous chemicals, making safety precautions and procedures essential to prevent accidents and injuries.',
  'Number': 3},
 {

In [ ]:
#evol = completion(evolve_question_prompt.format(question = base_questions[1]['Question'] , answer = base_questions[0]['Answer']))
#display(evol)

In [119]:
def answer_to_quesion(question):
    rag_results = col.query(query_texts=question,n_results=5)
    text = '\n'.join( rag_results['documents'][0] )
    ap = answer_prompt.format(question=question,text=text)
    res = completion(msg=ap,sys_msg="")
    return res
def get_not_found_questions(base_questions):
    not_found = []
    for row in base_questions:
        question = row['Question']
        number = row['Number']
        print(number , question)
        res = answer_to_quesion(question)
        print(res)
        if res['Answerable'] == False:
            res['Question']=question
            res['Graph_Answer']=row['Answer']
            not_found.append(res)
    return not_found



not_found = get_not_found_questions(base_questions)
not_found

[{'Answer': 'The text primarily focuses on soldering processes and doesn\'t explicitly mention key materials used in soldering. It does mention "solder" and "flux" as essential components. ',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some key materials used in soldering?',
  'Graph_Answer': 'Solder, typically an alloy of tin and lead, and protective coatings.'},
 {'Answer': 'The text does not mention specific industries where soldering is essential. It focuses on the soldering process itself and its applications in electronics assembly.',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some industries where soldering is essential?',
  'Graph_Answer': 'Electronics assembly, plumbing, and construction.'}]

In [72]:
col_questions = client.get_or_create_collection('questions')
col_questions.add(ids='1', documents=str(not_found))
col_questions.get()

In [76]:
not_found

[{'Answer': "The text discusses the importance of cleanliness in electronics assembly, especially in relation to soldering.  It highlights that contaminants from soldering processes, such as flux and flux residues, are a major factor in the need for cleaning.  However, it also emphasizes that other sources of contamination, like those from printed circuit boards and components, should not be overlooked. The text goes on to elaborate on the different types of contaminants and how they can affect the soldering process, but doesn't specifically mention how cleanliness impacts the soldering process.",
  'Refrences': 'CONTAMINATION 203\n\nAim of cleaning is simply to remove contamination. To do this effectively and choose the best cleaning agent, therefore, knowledge of contaminants involved is vital. In other words, what is on the assembly which needs to be removed by a cleaning process? There are three main sources of contaminant:\n@ printed circuit boards used in assembly manufacture\n@ 

In [52]:
question = base_questions[8]['Question']
print(question)
rag_results = col.query(query_texts=question,n_results=5)
rag_results

How does flux play a role in improving the soldering process?


{'ids': [['soldering.pdf_78',
   'soldering.pdf_76',
   'soldering.pdf_72',
   'soldering.pdf_71',
   'soldering.pdf_7']],
 'distances': [[0.5902811884880066,
   0.6446360349655151,
   0.6565366983413696,
   0.6579433679580688,
   0.6762923002243042]],
 'metadatas': [[None, None, None, None, None]],
 'embeddings': None,
 'documents': [['98 FLUX\n\nAPPLICATION OF FLUXES 99 Figare 5.5 Principle of brush Sluxing @ whether the assembly must be cleaned after soldering @ what cleaning process is used. Consequently, it’s not hard to appreciate; flux should be chosen to suit cleaning processes, rather than the other way round. Application of fluxes Methods of applying flux are manifold — there are almost as many methods as there are flux types. However, methods are conveniently split into whether assemblies to be fluxed are undergoing CS or SC soldering processes. CS soldering processes imply a liquid flux, while SC soldering processes indicate flux is in paste form (in a mixture with solder p

In [121]:
res = answer_to_quesion("describe soldering metrics")
print(res['Answer'])

Soldering metrics are used to assess the quality of soldered joints in electronics assembly. This includes factors like solderability, which refers to the ability of surfaces to be wetted with molten solder and form smooth fillets, and the presence of non-wetting and de-wetting, which indicate incomplete wetting and retracting of solder, respectively. Inspections are conducted under magnification to observe wetting, penetration, and the presence of non-wetting or de-wetting, and non-wetting can be difficult to distinguish on plated through-hole boards. De-wetting resembles water on a greasy surface, with solder retracting into globules and ridges, leaving thin solder coatings between them.


In [88]:
col_questions = client.get_or_create_collection('questions')

for i,report in enumerate(communities_reports_content[:10]):
    base_questions = completion(create_base_question_prompt.format(doc=report))
    not_found = get_not_found_questions(base_questions)
    print(not_found)
    col_questions.add(ids=str(i), documents=str(not_found))


    



0 What is the central process in the community discussed in the text?
{'Answer': 'The central process discussed in the text is soldering, a process used in the electronics industry to bond components together, forming electrical connections. Soldering serves two functions: mechanical support, holding components together, and electrical support, forming required electrical connections.', 'Refrences': 'Soldering, in principle though not in practice, is a reasonably straightfor- ward process, used in the electronics industry to bond components to- gether, forming one or more electrical connections. From this description, it’s easy to see that soldering serves two functions: @ mechanical support — holding components of an assembly together @ electrical support — forming required electrical connections within a circuit.', 'Answerable': True}
1 What is the purpose of flux in soldering?
{'Answer': 'Flux plays a crucial role in soldering by cleaning the surface of the metal to be soldered, all

Insert of existing embedding ID: 1
Add of existing embedding ID: 1


{'Answer': 'Flux plays a crucial role in the success of soldering by cleaning the surfaces of metals to be joined, removing oxides and contaminants, thus allowing the solder to wet the surfaces effectively and form a strong bond.', 'Refrences': 'The effectiveness  of  a  flux  in  removing  oxide  and  contaminants  from  the   surface  of  metals  is  known  as  its  activity.', 'Answerable': True}
[{'Answer': "The text doesn't explicitly mention why fluxing the printed circuit board is crucial in soldering processes. It focuses on various fluxing methods and their applications in different soldering processes, but doesn't explain the importance of fluxing itself. Therefore, I can't provide an answer to your question based on the provided text.", 'Refrences': [], 'Answerable': False, 'Question': 'Why is fluxing the printed circuit board crucial in various soldering processes?', 'Graph_Answer': 'Fluxing is essential to clean the surface and ensure proper wetting and bonding of the sold

In [104]:

all_json = col_questions.get()['documents']
all_json

['[{\'Answer\': \'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability. \', \'Refrences\': [], \'Answerable\': False, \'Question\': \'What are some of the essential materials used in the soldering process?\', \'Graph_Answer\': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder\'s wetting and flow."}, {\'Answer\': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn\'t specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.", \'Refrence

[{'Answer': 'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability.',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some of the essential materials used in the soldering process?',
  'Graph_Answer': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder's wetting and flow."},
 {'Answer': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn't specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.",
  'Refrences': [],
  '

In [115]:
import json_repair

all_questions =[]
for j in all_json:
    all_questions.extend( json_repair.loads(j) )

all_questions

[{'Answer': 'I apologize, but the provided text does not specify the essential materials used in the soldering process. It mainly discusses the overall process, safety aspects, and the importance of solderability.',
  'Refrences': [],
  'Answerable': False,
  'Question': 'What are some of the essential materials used in the soldering process?',
  'Graph_Answer': "Solder and flux are crucial materials. Solder is the material that melts and flows into the joint, while flux removes oxides and contaminants from the surfaces, improving the solder's wetting and flow."},
 {'Answer': "The provided text focuses on various assembly variations and soldering processes used in electronics manufacturing, but it doesn't specifically mention how soldering contributes to assembly variations. The text describes different assembly methods for through-hole and surface mounted components, including CS (components before solder) and SC (solder before components) soldering processes.",
  'Refrences': [],
  '